# Ensembles

En esta notebook vamos a cargar el modelo que mejor anduvo en la CV


In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import pandas as pd
import csv

df_dev = pd.read_table("../../../data/es/dev_es.tsv", index_col="id", quoting=csv.QUOTE_NONE)
df_train = pd.read_table("../../../data/es/train_es.tsv", index_col="id", quoting=csv.QUOTE_NONE)
df_test = pd.read_table("../../../data/es/test_es.tsv", index_col="id", quoting=csv.QUOTE_NONE)



X_train, y_train = df_train["text"], df_train["HS"]
X_dev, y_dev = df_dev["text"], df_dev["HS"]

print("Instancias de entrenamiento: {}".format(len(df_train)))
print("Instancias de desarrollo: {}".format(len(df_dev)))
print("Instancias de test: {}".format(len(df_test)))


Instancias de entrenamiento: 4500
Instancias de desarrollo: 500
Instancias de test: 1600


Hagamos tokenize sobre el texto

In [2]:
%%capture
from elmoformanylangs import Embedder

e = Embedder("../../../models/elmo/es/", batch_size=32)

In [3]:
from keras.optimizers import Adam
from keras.utils import plot_model
from hate.nn import CharModel, ElmoModel, BowModel, MergeModel

def create_model(params, embedder):
    params = params.copy()

    dropout = params.pop('dropout')
    recursive_class = params.pop('recursive_class')
    dense_last_layer = params.pop('dense_last_layer')
    char_model = CharModel(
        vocab_size=params.pop('char__vocab_size'),
        max_charlen=params.pop('char__max_charlen'),
        embedding_dim=params.pop('char__embedding_dim'),
        tokenize_args={
            "stem": params.pop('char__stem'),
            "alpha_only": params.pop('char__alpha_only'),
        },
        filters=params.pop('char__filters'),
        kernel_size=params.pop('char__kernel_size'),
        pooling_size=params.pop('char__pooling_size'),
        dense_units=dense_last_layer,
        recursive_class=recursive_class, dropout=dropout
    )
    elmo_model = ElmoModel(
        max_len=50, embedder=embedder,
        lstm_units=params.pop('elmo__lstm_units'),
        tokenize_args={'deaccent': params.pop('elmo__deaccent')},
        dense_units=dense_last_layer,
        recursive_class=recursive_class, dropout=dropout
    )
    bow_model = BowModel(
        num_words=params.pop('bow__num_words'),
        dense_units=[1024, dense_last_layer], dropout=dropout,
    )
    merge_model = MergeModel([char_model, elmo_model, bow_model])
    optimizer_args = {
        "lr": params.pop('lr'),
        "decay": params.pop('decay')
    }
    merge_model.compile(loss='binary_crossentropy',
              optimizer=Adam(**optimizer_args),
              metrics=['accuracy'])

    assert(len(params) == 0)

    return merge_model

Using TensorFlow backend.


Carguemos los hiperparámetros óptimos

In [4]:
import pickle

with open("../../../models/model_selection.pkl", "rb") as f:
    model_selection = pickle.load(f)
    
df = pd.DataFrame(model_selection)
df = df.sort_values("val_acc", ascending=False)
df.shape

(73, 6)

In [5]:
params = df.iloc[0]["params"]

params["lr"] = 0.00035
params["dropout"] = [0.80, 0.55]
params

{'recursive_class': keras.layers.cudnn_recurrent.CuDNNLSTM,
 'lr': 0.00035,
 'elmo__lstm_units': 256,
 'elmo__deaccent': False,
 'dropout': [0.8, 0.55],
 'dense_last_layer': 64,
 'decay': 0.01,
 'char__vocab_size': 150,
 'char__stem': False,
 'char__pooling_size': 4,
 'char__max_charlen': 200,
 'char__kernel_size': 7,
 'char__filters': 256,
 'char__embedding_dim': 128,
 'char__alpha_only': False,
 'bow__num_words': 3500}

In [6]:
from keras.utils import plot_model
from IPython.display import Image

model = create_model(params, e)

print(model.summary())


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Char_Input (InputLayer)         (None, 200)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 200, 128)     19200       Char_Input[0][0]                 
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 200, 256)     229632      embedding_1[0][0]                
__________________________________________________________________________________________________
max_pooling1d_1 (MaxPooling1D)  (None, 50, 256)      0           conv1d_1[0][0]                   
__________________________________________________________________________________________________
Elmo_Input

In [7]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpointer = ModelCheckpoint('/tmp/ensemble_17.h5', save_best_only=True, monitor='val_acc', verbose=1)
early_stopper = EarlyStopping(monitor='val_loss', patience=15)
history = model.fit(X_train, y_train, callbacks=[checkpointer, early_stopper],
          validation_data=(X_dev, y_dev), epochs=300, batch_size=32)


Train on 4500 samples, validate on 500 samples
Epoch 1/300
4500/4500 [==============================] - 7s 1ms/step - loss: 0.6924 - acc: 0.5733 - val_loss: 0.6527 - val_acc: 0.6620

Epoch 00001: val_acc improved from -inf to 0.66200, saving model to /tmp/ensemble_17.h5
Epoch 2/300
4500/4500 [==============================] - 5s 1ms/step - loss: 0.6324 - acc: 0.6516 - val_loss: 0.6164 - val_acc: 0.6780

Epoch 00002: val_acc improved from 0.66200 to 0.67800, saving model to /tmp/ensemble_17.h5
Epoch 3/300
4500/4500 [==============================] - 5s 1ms/step - loss: 0.5816 - acc: 0.7176 - val_loss: 0.5820 - val_acc: 0.7360

Epoch 00003: val_acc improved from 0.67800 to 0.73600, saving model to /tmp/ensemble_17.h5
Epoch 4/300
4500/4500 [==============================] - 5s 1ms/step - loss: 0.5419 - acc: 0.7413 - val_loss: 0.5481 - val_acc: 0.7500

Epoch 00004: val_acc improved from 0.73600 to 0.75000, saving model to /tmp/ensemble_17.h5
Epoch 5/300
4500/4500 [=========================

In [9]:
from hate.utils import print_evaluation

model.load_weights(checkpointer.filepath)
model.preprocess_fit(X_train)

print_evaluation(model, X_dev, y_dev)

500/500 [==============================] - 0s 398us/step
Loss        : 0.4117
Accuracy    : 0.8300
Precision   : 0.8128
Recall      : 0.8018
F1          : 0.8073


## Submission


In [15]:
df_test["preds"] = (model.predict(df_test["text"]) >= 0.5).astype(int)
df_dev["preds"] = (model.predict(df_dev["text"]) >= 0.5).astype(int)

In [16]:
!rm es_a.tsv

rm: cannot remove 'es_a.tsv': No such file or directory


In [17]:

with open('es_a.tsv', 'w') as f:
    for i, row in df_test.iterrows():
        f.write('{}\t{}\n'.format(i, row["preds"]))


In [18]:
!cp es_a.tsv ../../../submissions//17_ensemble.tsv 

In [19]:
import zipfile

with zipfile.ZipFile("es_a.tsv.zip", 'w') as f:
    f.write('es_a.tsv')
